# Ising model

We study the Hamiltonian of the transverse-field Ising model with periodic boundary conditions and transverse field `g` set to 1.

In [1]:
using CondensedMatterSOS
@spin σ[1:2]
ising_hamiltonian(σ, 1, true)

(2 + 0im)σˣ₁σˣ₂ + -σᶻ₁ + -σᶻ₂

Let's pick a solver from [this list](https://jump.dev/JuMP.jl/dev/installation/#Getting-Solvers).

In [2]:
using CSDP
solver = optimizer_with_attributes(
    () -> MOIU.CachingOptimizer(MOIU.UniversalFallback(MOIU.Model{Float64}()), CSDP.Optimizer()),
    MOI.Silent() => true
)

MathOptInterface.OptimizerWithAttributes(Main.##262.var"#1#2"(), Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.Silent() => true])

We can compute a lower bound `-2√2` to the ground state energy as follow:

In [3]:
function hamiltonian_energy(N, maxdegree, solver; kws...)
    @spin σ[1:N]
    H = ising_hamiltonian(σ, 1, true)
    energy(H, maxdegree, solver; kws...)
end
bound, gram, ν = hamiltonian_energy(2, 2, solver, sparsity = NoSparsity())
bound

-2.828427145137386

We can see that the moment matrix uses all monomials:

In [4]:
ν.basis.monomials

7-element Vector{CondensedMatterSOS.SpinMonomial}:
 1
 σˣ₁
 σʸ₁
 σᶻ₁
 σˣ₂
 σʸ₂
 σᶻ₂

Using term sparsity with block/cluster completion, we get the same bound:

In [5]:
bound, gram, ν = hamiltonian_energy(2, 2, solver)
bound

-2.828427141662865

But with a smaller basis:

In [6]:
[M.basis.monomials for M in ν.sub_moment_matrices]

2-element Vector{Vector{CondensedMatterSOS.SpinMonomial}}:
 [σᶻ₁, σᶻ₂, 1]
 [σˣ₁, σʸ₁, σˣ₂, σʸ₂]

Using term sparsity with chordal completion, we get a smaller bound:

In [7]:
bound, gram, ν = hamiltonian_energy(2, 2, solver, sparsity = MonomialSparsity(ChordalCompletion()))
bound

-4.000000003141185

But with an even smaller basis:

In [8]:
[M.basis.monomials for M in ν.sub_moment_matrices]

5-element Vector{Vector{CondensedMatterSOS.SpinMonomial}}:
 [σˣ₂, σʸ₂]
 [σᶻ₂, 1]
 [σᶻ₁, 1]
 [σˣ₁, σˣ₂]
 [σˣ₁, σʸ₁]

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*